<a href="https://colab.research.google.com/github/Edward-TL/VS_Buy/blob/master/VS_Buy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import


In [43]:
from google.colab import drive
drive.mount('/content/drive')

#For the system
import os
from plotly.offline import iplot

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time
import re

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

import plotly
import nltk

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [44]:
from dataclasses import dataclass

@dataclass
class Page:
    name:str
    '''Page index given by the backend'''
    index: int
    '''Page URl with name of the format replacers'''
    url : str      
    page_test : str
    '''Tuple with the name of the replacers'''
    url_replacers : tuple
    '''string that will replace the space of the users request on a search'''
    space_replacer : tuple
    '''All the next properties are tuples that contains:
    ("node: span", "attribute: class", "=predicate: text")'''
    boxes : tuple
    '''Products Info'''
    product_urls : tuple
    url_get : str
    # self.product_id = product_id
    name_and_images : tuple
    names_get : str
    images_get : str
    '''All the next properties are tuples that contains:
    ("node: span", "attribute: class", "=predicate: text")'''
    reviews : tuple
    stars : tuple
    price : tuple
    highlights : tuple
    '''dictionary'''
    money_dict : dict
    
    def adapt_url(self, Page, user_request, country_domain):
        check = str(type(Page.url_replacers))
        if country_domain[0] != ".":
            country_domain = '.' + country_domain
        
        
        if check != "<class 'str'>":
            adapted_url = Page.url.replace(Page.url_replacers[0], country_domain)
            for r in range(1, len(Page.url_replacers)):
                user_request_adapted = user_request.replace(' ', Page.space_replacer[r-1])
                adapted_url = adapted_url.replace(Page.url_replacers[r], user_request_adapted)
        else:
            user_request_adapted = user_request.replace(' ', Page.space_replacer[0])
            adapted_url = Page.url.replace(Page.url_replacers, user_request_adapted)
        
        return adapted_url

amazon_money_dict = {'mx' : {'coin' : '$',
                      'thousands': ',',
                      'decimal': '.',
                      'two_prices_sep' : ' - '}
}

Amazon = Page(name = 'Amazon',
    index = 2,
    url='https://www.amazon.com{country}/s?k={user_request}',
    page_test='https://www.amazon.com.mx/s?k=audifonos+inalambricos',
    url_replacers=('{country}', '{user_request}'),
    space_replacer=['+'],
    boxes=('div', 'data-component-type', 's-search-result'),
    highlights=('div', 'class', 'a-row a-badge-region'),
    url_get='href',
    product_urls=('a', 'class', 'a-link-normal a-text-normal'),
    name_and_images=('div', 'class', 'a-section aok-relative s-image-square-aspect'),
    names_get='alt',
    images_get='src',
    reviews=('span', 'class', 'a-size-base'),
    stars=('span', 'class', 'a-icon-alt'),
    price=('span', 'class', 'a-offscreen'),
    money_dict=amazon_money_dict,
    #product_id('data-asin')
    )

ebay_money_dict = {'mx' : {'coin' : 'MXN $',
                      'thousands': ' ',
                      'decimal': '.',
                      'two_prices_sep' : ' a '}
            }

Ebay = Page(name='Ebay',
    index = 3,
    url='https://www.ebay.com/sch/i.html?_nkw={user_request}',
    page_test='https://www.ebay.com/sch/i.html?_nkw=audifonos+inalambricos',
    url_replacers=('{user_request}'),
    space_replacer=['+'],
    boxes=('li', 'class', 's-item'),
    highlights=('span', 'class', 'ui-search-item__highlight-label__text'),
    product_urls=('div', 'class', 's-item__image'),
    url_get='href',
    name_and_images=('div', 'class', 's-item__image-wrapper'),
    names_get='alt',
    images_get='src',
    reviews=None,
    stars=None,
    price=('span', 'class', 's-item__price'),
    money_dict=ebay_money_dict,)

meli_money_dict = {'mx' : {'coin' : '$',
                      'thousands': ',',
                      'decimal': '.',
                      'two_prices_sep' : ' - '}
            }
            
Mercado_Libre = Page(name='Mercado Libre',
    index = 4,
    url='https://listado.mercadolibre.com{country}/{user_request_1}#D[A:{user_request_2}]',
    page_test='https://listado.mercadolibre.com.mx/audifonos-inalambricos#D[A:audifonos%20inalambricos]',
    url_replacers=('{country}', '{user_request_1}', '{user_request_2}'),
    space_replacer=['-','%20'],
    boxes=('div', 'class', 'ui-search-result__wrapper'),
    highlights=('span', 'class', 'ui-search-item__highlight-label__text'),
    product_urls=('div', 'class', 'ui-search-result__image'),
    url_get='href',
    name_and_images=('div', 'class', 'slick-slide slick-active'),
    names_get='alt',
    images_get='data-src',
    reviews=None,
    stars=None,
    price=('span', 'class', 'price-tag ui-search-price__part'),
    money_dict=meli_money_dict,)

# wallmart_money_dict = {'mx' : {'coin' : '$',
#                       'thousands': ',',
#                       'decimal': '.',
#                       'two_prices_sep' : ' - '}
# }

# Wallmart = Page(name = 'Wallmart',
#     url='https://www.walmart.com{country}/productos?Ntt={user_request}',
#     url_replacers=('{country}', '{user_request}'),
#     space_replacer=['%20'],
#     boxes=('div', 'class', 'grid_product__30OQa'),
#     highlights=None,
#     product_urls=('a', 'class', 'nav-link_navLink__2oJ29 product_image__LcsAj'),
#     url_get='href',
#     name_and_images=('span', 'class', 'lazyload-wrapper'),
#     names_get='alt',
#     images_get='src',
#     reviews=None,
#     stars=None,
#     price=('p', 'data-automation-id', 'sale-price'),
#     money_dict=wallmart_money_dict,
#     #product_id('data-asin')
#     )

best_buy_money_dict = {'mx' : {'coin' : '$',
                      'thousands': ',',
                      'decimal': '.',
                      'two_prices_sep' : ' - '}
}

Best_Buy = Page(name = 'Best Buy',
    index = 7,
    url='https://www.bestbuy.com{country}/c/buscar-best-buy/buscar?query={user_request}',
    page_test='https://www.bestbuy.com.mx/c/buscar-best-buy/buscar?query=audifonos+inalambricos',
    url_replacers=('{country}', '{user_request}'),
    space_replacer=['+'],
    boxes=('div', 'class', 'product-line-item-line'),
    highlights=None,
    product_urls=('div', 'class', 'product-title'),
    url_get='href',
    name_and_images=('div', 'class', 'col-xs-3 image-container'),
    names_get='alt',
    images_get='src',
    reviews=None,
    stars=None,
    price=('div', 'class', 'product-price'),
    money_dict=best_buy_money_dict,
    #product_id('data-asin')
    )

Pages = [Amazon, Mercado_Libre, Ebay, Best_Buy]

class headers:
    wallmart = {'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                'accept-encoding':'gzip, deflate, br',
                'accept-language':'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
                'cache-control':'max-age=0',
                'upgrade-insecure-requests':'1',
                'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}

    h0 = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;x64; rv:66.0) Gecko/20100101 Firefox/66.0",
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1",
            "Connection":"close",
            "Upgrade-Insecure-Requests":"1"}

    h1 = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5)AppleWebKit/605.1.15 (KHTML, like Gecko)Version/12.1.1 Safari/605.1.15",
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1",
            "Connection":"close",
            "Upgrade-Insecure-Requests":"1"}

    h2 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1",
            "Connection":"close",
            "Upgrade-Insecure-Requests":"1"}

    common = (h0, h2, h2)
    h3 = wallmart
    all_saved = (h0, h2, h2, wallmart)


# Scrape Funcs

In [45]:
#Web Manage
from bs4 import BeautifulSoup
import requests
import re

def extract_soup(url, header=0, just_status=False, just_soup=False):
    header = headers.all_saved[header]
    response = requests.get(url, headers=header)
    status = response.status_code

    soup = BeautifulSoup(response.text, 'lxml')

    if just_status==True:
        return status
    elif just_soup==True:
        return soup
    else:
        return soup, status

def search_boxes(soup, box_tuple, cents=False):
    #Search the kind of box specified by the touple
    if cents == True:
        boxes = soup.find(box_tuple[0], attrs={box_tuple[1] : box_tuple[2]})
    else:
        boxes = soup.find_all(box_tuple[0], attrs={box_tuple[1] : box_tuple[2]})

    return boxes

def get_brute_info(boxes_array, info_tuple):
    '''Returns the brute text from the html depending of the info you asked'''
    info = [None]*len(boxes_array)
    searcher = None
    i=0
    for box in boxes_array:
        searcher = search_boxes(box, info_tuple)
        info[i] = searcher
        searcher = None
        i += 1
    
    return info


# Page getters

In [46]:
def get_names(boxes_array, Page, test_all=False, test_len=False, position=None):
    names = [None]*len(boxes_array)
    name = Page.names_get

    if test_all == True:
            print(f'Boxes array len: {len(boxes_array)}')

    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position != None:
        searcher = search_boxes(boxes_array[position], Page.name_and_images)
        if searcher:
            if Page.name == 'Best Buy':
                image_name = searcher[0].img.get(name)
                name_split = image_name.split(' - ')
                names = name_split[1]
            else:
                names = searcher[0].img.get(name)
    else:
        '''For Testing the functions and Xpaths'''
        b=0
        if test_all == True:
            print('For Loop')
        for box in boxes_array:
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.name_and_images)
            if searcher:
                # try:
                if Page.name == 'Best Buy':
                    image_name = searcher[0].img.get(name)
                    name_split = image_name.split(' -')
                    if len(name_split) > 1:
                        names[b] = name_split[1].lstrip()
                    if len(name_split) == 1:
                        names[b] = name_split[0].lstrip()
                else:
                    names[b] = searcher[0].img.get(name)
                # except:
                #     name_message_error = f'''Value info:
                #     box: {box}
                #     Searcher: {searcher}
                #     Searcher[0].img = {searcher[0].img}'''

                #     raise ValueError(name_message_error)
                
            b +=1
    if test_all == True:
        print(names)
    
    elif test_len == True:
        print(len(names))

    return names


def get_images(boxes_array, Page, test_all=False, test_len=False, position=None):
    images = [None]*len(boxes_array)
    image = Page.images_get
    
    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position:
        searcher = search_boxes(boxes_array[position], Page.name_and_images)

        if searcher:
            images = searcher[0].img.get(image)
    else:
        '''For Testing the functions and Xpaths'''
        b=0
        for box in boxes_array:
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.name_and_images)
            if searcher:
                images[b] = searcher[0].img.get(image)
            b +=1

    if test_all == True:
        print(images)
    
    elif test_len == True:
        print('images:', len(images))

    return images

def get_products_urls(boxes_array, Page, test_all=False, test_len=False, position=None):
    urls = [None]*len(boxes_array)
    url = Page.url_get
    if test_all == True:
        print('Urls:')
    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position:
        searcher = search_boxes(boxes_array[position], Page.product_urls)
        if searcher:
            try:
                if Page.name == 'Amazon':
                    source_url = searcher[0].get(url)
                    position_url = 'https://www.amazon.com.mx' + source_url
                    
                else:
                    position_url = searcher[0].a.get(url)
                
                urls = position_url
            except:
                error_message = f'''Value info:
                Searcher: {searcher}
                searcher[0]: {searcher}
                searcher[0].a: searcher[0].a
                url: {Page.url_get}
                position: {position}
                '''

                raise ValueError(error_message)
    else:
        '''For Testing the functions and Xpaths'''
        b=0
        for box in boxes_array:
            if test_all == True:
                print('For loop:')
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.product_urls)
            if searcher:
                if test_all == True:
                    print(searcher[0].a)
                if Page.name == 'Amazon':
                    source_url = searcher[0].get(url)
                    urls[b] = 'https://www.amazon.com.mx' + source_url
                    
                else:
                    urls[b] = searcher[0].a.get(url)
            b +=1

    if test_all == True:
        print(urls)
    
    elif test_len == True:
        print('urls:', len(urls))
    return urls

def get_price(country, boxes_array, Page, test_all=False, test_len=False, position=None):
    price = [None]*len(boxes_array)
    coin_symbol = Page.money_dict[country]['coin']
    k_sep = Page.money_dict[country]['thousands']
    d_sep = Page.money_dict[country]['decimal']
    tps = Page.money_dict[country]['two_prices_sep']
    price_string = 'start'

    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position:
        searcher = search_boxes(boxes_array[position], Page.price)
        if searcher:
            try:
                price_string = searcher[0].get_text().split(tps)
                price_string = price_string[0].replace(coin_symbol,'').replace(k_sep,'').replace(d_sep,'.')
                #Special case
                price_string = re.findall(r'(\d+\.\d+)', price_string)
                price = float(price_string[0])
            except:
                error_message = f'''String index out of range. 
                Money dictionary: {Page.money_dict}
                Original String: {searcher[0].get_text()}
                Box #{position}'''
                raise ValueError(error_message)     
        
    #For Testing the functions and Xpaths
    else:
        b=0
        for box in boxes_array:
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.price)
            if test_all == True:
                print(searcher)
            if searcher:
                if country == 'mx':
                    try:
                        price_string = searcher[0].get_text().split(tps)
                        if test_all == True:
                            print(price_string)
                        price_string_bfre = price_string[0].replace(coin_symbol,'').replace(k_sep,'').replace(d_sep,'.')
                        if test_all == True:
                            print(price_string) 
                        #Ebays Special case
                        price_string_bfre = price_string_bfre.replace('\xa0','')
                        #Just in case
                        price_string_check = re.findall(rf"(\d+\.?\d+)", price_string_bfre)
                        price[b] = float(price_string_check[0])                        

                    except:
                        error_message = f'''Info about the Value. 
                        Money dictionary: {Page.money_dict}
                        Original String: {searcher[0].get_text()}
                        Before RegEx: {price_string_bfre}
                        Type Before RegEx: {type(price_string)}
                        Price string: {price_string}
                        Box #{b}'''
                        raise ValueError(error_message)                        
            b +=1

    if test_all == True:
        print('prices:', len(price), price)
    
    elif test_len == True:
        print('prices:', len(price))

    return price

def get_stars(boxes_array, Page, country='mx', test_all=False, test_len=False, position=None):
    stars = [None]*len(boxes_array)
    decimal_sep = Page.money_dict[country]['decimal']

    b=0
    for box in boxes_array:
        #Remember that boxes are arrays
        searcher = search_boxes(box, Page.stars)

        if searcher:
            if decimal_sep == '.':
                stars[b] = float(searcher[0].get_text()[:3])
            else:
                stars[b] = float(searcher[0].get_text()[:3].replace(decimal_sep,''))

        b +=1
    if test_all == True:
        print(stars)
    
    elif test_len == True:
        print('stars:', len(stars))
    return stars

def get_reviews(boxes_array, Page, country='mx', test_all=False, test_len=False, position=None):
    reviews = [None]*len(boxes_array)
    comma_sep = Page.money_dict[country]['thousands']

    b=0
    for box in boxes_array:
        #Remember that boxes are arrays
        searcher = search_boxes(box, Page.reviews)

        if searcher:
            if len(searcher) > 1:
                searcher = [searcher[0]]

            try:
                reviews[b] = int(searcher[0].get_text().replace(comma_sep,''))
            except:
                pass
        
        b +=1
    if test_all == True:
        print(reviews)
    
    elif test_len == True:
        print('reviews:', len(reviews))
    return reviews

def amazon_products_id(boxes_array, test=False):
    ids = [None]*len(boxes_array)

    b=0
    for box in boxes_array:
        #Remember that boxes are arrays
        if box:
            product_id = box.get('data-asin')
            ids[b] = 'www.amazon.com.mx/dp/' + product_id
            
        b +=1
    if test == True:
        print(ids)
    return ids



# Cheapest Functions

In [47]:
def ordered_dict(dictionary, reverse=True):
    import operator
    ordered_dict = dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))

    return ordered_dict

def cheapest(array_prices, just_position=True, just_price=False, position_and_price=False, test=False):
    cheapest_position = 0
    cheapest_price = None
    
    for n in range(len(array_prices)):
        price = array_prices[n]
        if price:
            if cheapest_price == None:
                cheapest_price = price
            if test == True:
                print(f'number = {n} cheapest_price is {cheapest_price} and price check is {price}')

            if price < cheapest_price:
                if test == True:
                    print(f'price: {price} < {cheapest_price}')

                cheapest_price = price
                cheapest_position = n
        
    if position_and_price == True:
        just_position = False
        return cheapest_position, cheapest_price
    elif just_position == True:
        return cheapest_position
    elif just_price == True:
        return cheapest_price


def get_cheapest(cheapest_idx, products, cheapest_price=None, country=None, Page=None):
    products_type = str(type(products))
    if products_type == "<class 'bs4.element.ResultSet'>":
        
        if country and Page:
            cheapest_dict = {}
            cheapest_dict['name'] = get_names(products, Page, position=cheapest_idx)
            cheapest_dict['image'] = get_images(products, Page, position=cheapest_idx)
            cheapest_dict['url'] = get_products_urls(products, Page, position=cheapest_idx)
            cheapest_dict['price'] = cheapest_price
        else:
            if not country:
                raise ValueError("Missing country value")
            elif not Page:
                raise ValueError("Missing Page object")
            elif not cheapest_price:
                raise ValueError("Missing cheapest_price value (int)")
            else:
                raise ValueError(f'''Missing Page and country values.
            You can use this function with a dictionary with all the data preloaded as well''')
            
    elif products_type == "<class 'dict'>":
        cheapest_dict = {}
        products_dictionary = products
        for key in products_dictionary:
            cheapest_dict[key] = products_dictionary[key][cheapest_idx]
    else:
        error_in_product_type =  f"Products type must be a dict or a bs4.element.ResultSet with the boxes. Recived {type(products)}"
        raise ValueError(error_in_product_type)
    
    return cheapest_dict

# Main

In [48]:
def scrap_cheapest(user_request, Page, country='mx'):
    #Adapt the url
    request_url = Page.adapt_url(Page, user_request, country)
    #All the HTML of the page
    soup_request, status = extract_soup(request_url, 1)
    if status == 503:
        while status == 503:
            time.sleep(1)
            soup_request, status = extract_soup(request_url)
            log_status[n_log] = status
            log_date[n_log] = datetime.now()

    if status ==200:
        # #HTML divided by products, and stored as elements of an array
        page_boxes = search_boxes(soup_request, Page.boxes)
        # From this part, could get better AFTER the 4 scrapers are made
        #From the Boxes, obtain the prices
        price_boxes = get_price(country, page_boxes, Page)
        #Obtain the cheapest from prices and then, you obtain the cheapest product as a dictionary
        cheapest_idx, cheapest_price = cheapest(price_boxes, position_and_price=True)
        cheapest_product_dictionary = get_cheapest(cheapest_idx, page_boxes, cheapest_price, country, Page)

        return cheapest_product_dictionary
    else:
        pass

In [49]:
# user_request = 'audifonos inalambricos'
# pages = [Amazon, Mercado_Libre, Ebay, Best_Buy]
# pages = [Amazon]
# for page in pages:
#     print(f'Searching {user_request} in -> {page.name}')
#     cheapest_dict = scrap_cheapest(user_request, page)
#     for key in cheapest_dict:
#         print(key,':', cheapest_dict[key])
#     print('\n')

# Name match test

In [50]:
user_request = 'audifonos inalambricos Bluetooth Marshall Major III'
country = 'mx'
def request_products(user_request, Page, header, country='mx'):
    page_url = Page.adapt_url(Page, user_request, country)

    #All the HTML of the page
    
    page_soup, status = extract_soup(page_url, header)
    if status == 503:
        while status == 503:
            time.sleep(1)
            page_soup, status = extract_soup(page_url)
    elif status == 200:
        # #HTML divided by products, and stored as elements of an array
        page_boxes = search_boxes(page_soup, Page.boxes)
        # print(page_boxes)
        page_products = {}

        page_products['names'] = get_names(page_boxes, Page)
        # #Best_Buy's images source (link)
        page_products['images'] = get_images(page_boxes, Page)
        page_products['urls'] = get_products_urls(page_boxes, Page)
        page_products['prices'] = get_price(country, page_boxes, Page)
        page_products['status'] = status

        return page_products
    else:
        print(status)
        page_products = {}
        page_products['store'] = Page.name
        page_products['idx'] = Page.index
        page_products['product'] = user_request
        page_products['names'] = []
        # #Best_Buy's images source (link)
        page_products['images'] = []
        page_products['urls'] = []
        page_products['prices'] = []
        page_products['status'] = status
        
        return page_products

In [51]:
def scrap_cheapest(user_request, Page, country='mx'):
    #Adapt the url
    request_url = Page.adapt_url(Page, user_request, country)
    #All the HTML of the page
    soup_request = extract_soup(request_url, 1, just_soup=True)
    # #HTML divided by products, and stored as elements of an array
    page_boxes = search_boxes(soup_request, Page.boxes)
    # From this part, could get better AFTER the 4 scrapers are made
    #From the Boxes, obtain the prices
    price_boxes = get_price(country, page_boxes, Page)
    #Obtain the cheapest from prices and then, you obtain the cheapest product as a dictionary
    cheapest_idx, cheapest_price = cheapest(price_boxes, position_and_price=True)
    cheapest_product_dictionary = get_cheapest(cheapest_idx, page_boxes, cheapest_price, country, Page)

    return cheapest_product_dictionary

## Name Hits

In [52]:
import unicodedata
def standarize(word):
    word = word.lower()
    word = unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore')
    return word

def name_hits(user_request, products_dict, array=True, dictionary=False):
    request_splited = user_request.split()
    if dictionary == True:
        hits_dict = {}
    if array == True:
        hits_array = []
    if len(products_dict['names']) > 0 and products_dict['names'] != None:
        for product_name in products_dict['names']:
            points = 0
            if product_name != None:
                product_splited = product_name.split()

                n = 0
                word = request_splited[n]
                for split in product_splited:
                    split = standarize(split)
                    while word != split and n < len(request_splited):
                        word = standarize(request_splited[n])

                        if word == split:
                            points += 1
                        n += 1
                    n = 0
            else:
                points = 0
                
            if dictionary == True:
                hits_dict[product_name] = points/len(request_splited)
            if array == True:
                hits_array.append(points/len(request_splited))

    if dictionary == True and array == False:
        return hits_dict
    elif array == True and dictionary == False:
        return hits_array
    else:
        return hits_dict, hits_array



# Data landing

In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from plotly.offline import iplot
import plotly.graph_objs as go

## Clasification

### Stads

In [54]:
def mean(array):
    summatory = 0
    n = 0
    for x in array:
        if x != None:
            summatory = summatory + x
            n += 1
    return summatory/n

def variance(array):
    average =  mean(array)
    s_sum = 0
    for x in array:
        if x != None:
            dif = x - average
            s_dif = dif**2
            s_sum = s_sum + s_dif
    n = len(array)

    return s_sum/n

def std(array):
    var = variance(array)
    
    return var**(1/2)

def max_with_none(array):
    if len(array) > 0 or array != None:
        max = 0
        for x in array:
            if x != None:
                if max < x:
                    max = x
        return max
    else:
        return None

def min_with_none(array):
    if len(array) > 0:
        n = 0
        min = None
        while min == None and n < len(array):
            min = array[n]
            n += 1
            
        for x in array:
            if x != None and min != None:
                if x < min:
                    min = x
        return min

    else:
        return None


def max_and_min(array):
    max = 0
    min = None
    n = 0
    while min == None and n < len(array):
        min = array[n]
        n += 1

    for x in array:
        if x != None:
            try:
                if max < x:
                    max = x
            except:
                max_error = f"'<' not supported between instances of 'float' and 'NoneType'. max = {max} | x = {x}"
                raise TypeError(max_error)
            try:
                if x < min:
                    min = x
            except:
                min_error = f"'<' not supported between instances of 'float' and 'NoneType'. x = {x} | min = {min}"
                raise TypeError(min_error)

    return max, min

In [55]:
test = [13, 22, 26, 38, 36, 42,49, 50, 77, 81, 98, 110]
print(mean(test))
print(variance(test))
print(std(test))

53.5
881.75
29.694275542602483


### Corpus

In [56]:
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

stopwords = set(stopwords.words('spanish'))

def create_corpus_points(user_request, Pages_array, header):
    corpus = {}
    pattern = r'''(?x)                  # Flag para iniciar el modo verbose
              (?:[A-Z]\.)+            # Hace match con abreviaciones como U.S.A.
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              | \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              | \.\.\.              # Hace match con puntos suspensivos
              | [][.,;"'?():-_`]    # Hace match con signos de puntuación
'''
    for Page in Pages:
        page_url = Page.adapt_url(Page, user_request, country)
        #All the HTML of the page
        page_soup = extract_soup(page_url, header, just_soup=True)
        # #HTML divided by products, and stored as elements of an array
        page_boxes = search_boxes(page_soup, Page.boxes)
        # print(page_boxes)
        page_products = {}
        page_names = get_names(page_boxes, Page)
        points = len(page_names)
        for product in page_names:
            if product != None:
                # words = nltk.regexp_tokenize(product, pattern)
                words = product.split()
                
                for word in words:
                    if word in corpus:
                        last_points = corpus[word]
                        total_points = last_points + points
                        corpus[word] = total_points
                        #thinkf of sum poinst of relevance by page
                    else:
                        corpus[word] = points
                points -= 1

    return corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Hit points by corpus

In [57]:
def hit_points_info(products_dict, corpus_points, array=True, dictionary=False):
    pattern = r'''(?x)                  # Flag para iniciar el modo verbose
              (?:[A-Z]\.)+            # Hace match con abreviaciones como U.S.A.
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              | \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              | \.\.\.              # Hace match con puntos suspensivos
              | [][.,;"'?():-_`]    # Hace match con signos de puntuación
'''
    minus_words = ('cambio', 'descompuesta')
    if dictionary == True:
        hits_dict = {}
    if array == True:
        hits_array = []
    if len(products_dict['names']) > 0 and products_dict['names'] != None:
        for product_name in products_dict['names']:
            if product_name != None:
                # product_name_words = nltk.regexp_tokenize(product_name, pattern)
                product_name_words = product_name.split()
                total_points = 0
                for word in product_name_words:
                    if word in minus_words:
                        total_points = total_points - 1000
                    else:
                        if word in corpus:
                            word_points = corpus[word]
                            total_points = total_points + word_points
                        else:
                            pass
                    
            else:
                total_points = 0
                
            if dictionary == True:
                hits_dict[product_name] = total_points
            if array == True:
                hits_array.append(total_points)

    if dictionary == True and array == False:
        return hits_dict
    elif array == True and dictionary == False:
        return hits_array
    else:
        return hits_dict, hits_array


### Main clas

In [94]:
def params(x, y, method_x, method_y, x_adjustment, products_dict):
    x = products_dict['prices']
    y = products_dict['hit points']
    if method_x == 'Stads':
        price_std_less = mean(x) - 1*std(x)
        if price_std_less < 0:
            max_x, min_x = max_and_min(x)
            price_std_plus = max_x
            price_mean = max_x/2
            price_std_less = min_x
        else:
            price_std_plus =  mean(x) + 1*std(x)
            price_mean = mean(x)

    if method_y == 'Stads':
        hit_emergency = False
        hits_std_plus = mean(y) + 1*std(y)
        hits_mean = mean(y)
        hits_std_less = mean(y) - 5*std(y)
        

        if hits_std_plus == 0 and hits_mean == 0 and hits_std_less == 0:
            hit_emergency = True
        else:
            hit_emergency = False
    
    if method_x == 'Max':
        max_x, min_x = max_and_min(x)
            
        price_std_plus = max_x * x_adjustment
        price_mean = (max_x + min_x)/2
        price_std_less = min_x + (max_x * (1-x_adjustment))

    if method_y == 'Max':
        hit_emergency = False

        max_y, min_y = max_and_min(y)
        hits_std_plus = max_y
        hits_std_less = min_y

        hits_mean = (max_y + min_y)/2
        

    params = {'price_std_plus' : price_std_plus,
                'price_mean' : price_mean,
                'price_std_less' : price_std_less,
                'hits_std_plus' : hits_std_plus,
                'hits_mean' : hits_mean,
                'hits_std_less' : hits_std_less,
                'hit_emergency' : hit_emergency,
                }
    return params

def categories(products_dict, params, method_y, colors_dict):
    colors = ['None'] * len(products_dict['names'])
    clasification = ['None'] * len(products_dict['names'])
    sizes = [15] * len(products_dict['names'])

    #Established the cheapest storage info
    cheapest_stock = {'Desireable': [None, None],
                      'Expensive but Requested': [None, None],
                      "Couldn't find better": [None, None],
                      'Emergency Case': [None, None],
                      'Ignored': [None, None],
                      }

    #For the parameters get before
    hit_emergency = params['hit_emergency']
    if method_y == 'Max':
        hit_param_top = hit_param_low = params['hits_mean']
    elif method_y == 'Stads':
        hit_param_top, hit_param_low = params['hits_std_plus'], params['hits_std_less']

    price_std_plus = params['price_std_plus']
    price_mean = params['price_mean']
    price_std_less = params['price_std_less']

    for product in range(len(products_dict['names'])):
        # if product == cheapest_idx:
        #     category = 'cheep_idx_choose'
        #     sizes[product] = 30
        # else:
        price = products_dict['prices'][product]
        hit = products_dict['hit points'][product]

        if price != None :

            if price >= price_mean and price <= price_std_plus:
                if hit > hit_param_top or hit_emergency == True:
                    category = 'Expensive but Requested'
                elif (hit <= hit_param_top) or (method_y == 'Stads' and hit >= hit_param_low and hit <= hit_param_top):
                    category = 'Emergency Case'
                
                else:
                    category = "Ignored"
                
            elif price <= price_mean and price >= price_std_less:
                if hit >= hit_param_top or hit_emergency == True:
                    category = 'Desireable'
                
                elif (hit <= hit_param_top) or (method_y == 'Stads' and hit >= hit_param_low and hit <= hit_param_top):
                    category = "Couldn't find better"
                
                else:
                    category = "Ignored"
            else:
                    category = "Ignored"
        else:
            category = "Ignored"
        

        if price != None:
            stock_price = cheapest_stock[category][0]

            if stock_price == None:
                cheapest_stock[category][0] = price
                cheapest_stock[category][1] = product
            else:
                if price < stock_price:
                    cheapest_stock[category][0] = price
                    cheapest_stock[category][1] = product

        clasification[product] = category
        colors[product] = colors_dict[category]
    
    return clasification, colors, cheapest_stock, sizes

def add_clasification_info(products_dict, method_x='Max', method_y='Max', x_adjustment=None,
 y_adjustment=None, cheapest_dict=False, graph=True, category_given=None):
    
    if x_adjustment == None:
        x_adjustment = 0.85
    if x_adjustment == None:
        x_adjustment = 1

    colors_dict = {'Ignored' : 'Black',
               'areas_choose':'LawnGreen',
                'Desireable': 'Blue',
                "Expensive but Requested": 'Green',
                "Couldn't find better": 'Gold',
                "Emergency Case": 'Crimson'
                }
    x = products_dict['prices']
    y = products_dict['hit points']
    
    param = params(x, y, method_x, method_y, x_adjustment, products_dict)
    clasification, colors, cheapest_stock, sizes = categories(products_dict, param, method_y, colors_dict)

    areas = iter(cheapest_stock.items())
    while True:
        try: 
            area = next(areas)
            if area[1][0] != None:
                category_given = area[0]
                raise StopIteration
        except StopIteration:
            break
    
    # if category_given == None:
    #     if cheapest_stock['Desireable'][0] != None:
    #         chosen_area = 'Desireable'
    #     else:
    #         if cheapest_stock['Expensive but Requested'][0] != None:
    #             chosen_area = 'Expensive but Requested'
    #         else:
    #             if cheapest_stock["Couldn't find better"][0] != None:
    #                 chosen_area = "Couldn't find better"
    #             else:
    #                 if cheapest_stock["Couldn't find better"][0] != None:
    #                     chosen_area = 'Emergency Case'
    #                 else:
    #                     chosen_area = 'Ignored'

    # else:
    chosen_area = category_given
    # try:

    print(chosen_area)
    areas_choose_product = cheapest_stock[chosen_area][1]
    print(cheapest_stock)
    if cheapest_dict == True:
        page_request_dict = {}
        page_request_dict['name'] = products_dict['names'][areas_choose_product]
        page_request_dict['picture'] = products_dict['images'][areas_choose_product]
        page_request_dict['url'] = products_dict['urls'][areas_choose_product]
        page_request_dict['price'] = products_dict['prices'][areas_choose_product]

    if graph == True:
        clasification[areas_choose_product] = 'areas_choose'
        colors[areas_choose_product] = colors_dict['areas_choose']
        sizes[areas_choose_product] = 30
    # except:
    #     error_message = f'''
    #         list indices must be integers or slices, not NoneType
    #         Names = {products_dict['names']}
    #         Prices = {products_dict['prices']}
    #         Hit Points = {products_dict['hit points']}
    #         Cheapest stock Dictionary = {cheapest_stock}'''
    #     raise TypeError(error_message)

        products_dict['price_std_plus'] = param['price_std_plus']
        products_dict['price_mean'] = param['price_mean']
        products_dict['price_std_less'] = param['price_std_less']
        products_dict['hits_std_plus'] = param['hits_std_plus']
        products_dict['hits_mean'] = param['hits_mean']
        products_dict['hits_std_less'] = param['hits_std_less']
        products_dict['colors'] = colors
        products_dict['clasification'] = clasification
        products_dict['sizes'] = sizes
        products_dict['cheapest_stock'] = cheapest_stock

    if cheapest_dict == True and graph == False:
        return page_request_dict
    elif graph == True and cheapest_dict == False:
        return products_dict
    elif cheapest_dict == True and graph == True:
        return page_request_dict, products_dict
    else:
        raise AttributeError('None value to give, page_request and graph are "False". Select one')

## Graph

In [59]:
def test_plot(products_dict, Page, user_request, rect_opacity=0.3, method_x='Max', method_y='Max'):
    trace = go.Scatter(
        x = products_dict['prices'],
        y = products_dict['hit points'],
        text = products_dict['names'],
        mode = 'markers',
        name = f'Results of web Scraper {Page.name}',
        marker = dict(color = products_dict['colors'],
                      size = products_dict['sizes'])
    )

    colors = products_dict['colors']
    clasification = products_dict['clasification']


    areas, x_mid_left, x_mid_right, y_mid_up, y_mid_down = areas_dicts(method_y, products_dict, rect_opacity)

    layout = {
        'title' : f"{Page.name}'s search result for {user_request}: Relevance vs Price",
        'xaxis_title' : 'Price MXN',
        'yaxis_title' : 'hit points',
        'shapes': areas
    }

    
    rect_labels = go.Scatter(
        x = [x_mid_left, x_mid_right, x_mid_left, x_mid_right],
        y = [y_mid_up, y_mid_up, y_mid_down, y_mid_down],
        text = ['Desireable', "Expensive & Requested", "Couldn't find better", "Emergency Case"],
        mode = 'text',
        name = 'Safe Areas'
    )

    fig = {
        'data': [trace, rect_labels],
        'layout': layout,
    }
    iplot(fig)

def areas_dicts(method_y, products_dict, rect_opacity):
    hits_std_plus = products_dict['hits_std_plus']
    hits_mean = products_dict['hits_mean']
    hits_std_less = products_dict['hits_std_less']
    
    price_std_plus = products_dict['price_std_plus']
    price_mean = products_dict['price_mean']
    price_std_less = products_dict['price_std_less']

    # to_print = ['price_std_plus', 'price_mean', 'price_std_less', 'hits_std_plus', 'hits_mean', 'hits_std_less']
    # for key in to_print:
    #     print(key,':', products_dict[key])
    if method_y == 'Max':
        y_top = hits_std_plus
        y_mid = hits_mean
        y_low = hits_std_less
    else:
        y_top = 1
        y_mid = hits_std_plus
        y_low = hits_std_less

    areas = [
        # Deseareable
        dict(
            type = "rect", xref = "x", yref = "y",
            x0 = price_std_less, y0 = y_top,
            x1 = price_mean, y1 = y_mid,
            opacity = rect_opacity, fillcolor = "LightSkyBlue", layer = 'below', line_width = 0
        ),
        # Expensive but Requested
        dict(
            type = "rect", xref = "x", yref = "y",
            x0 = price_mean, y0 = y_top,
            x1 = price_std_plus, y1 = y_mid,
            opacity = rect_opacity, fillcolor = "DarkCyan", layer = 'below', line_width = 0
        ),
        #Couldn't find better
        dict(
            type = "rect", xref = "x", yref = "y",
            x0 = price_std_less, y0 = y_mid,
            x1 = price_mean, y1 = y_low,
            opacity = rect_opacity, fillcolor = "DarkKhaki", layer = 'below', line_width = 0
        ),
        #Emergency Case
        dict(
            type = "rect", xref = "x", yref = "y",
            x0 = price_mean, y0 = y_mid,
            x1 = price_std_plus, y1 = y_low,
            opacity = rect_opacity, fillcolor = "DarkRed", layer = 'below', line_width = 0
        )
    ]

    x_mid_left = (price_std_less + price_mean) / 2
    x_mid_right = (price_mean + price_std_plus)/2

    y_mid_up = (y_top + y_mid) / 2
    y_mid_down = (y_mid + y_low)/2

    return areas, x_mid_left, x_mid_right, y_mid_up, y_mid_down

In [60]:
%cd 'drive/My Drive'
%cd 'Proyecto Final'

[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive/Proyecto Final
[Errno 2] No such file or directory: 'Proyecto Final'
/content/drive/My Drive/Proyecto Final


In [61]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [62]:
def get_df(sheet, products=False):
    wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1zwQ32di9dizxWFpM-fUo7wVAScVFHiiUdhdYe9-udpU/edit?usp=sharing")
    sheet = wb.worksheet(sheet)
    data = sheet.get_all_values()
    df = pd.DataFrame(data)
    df.columns = df.iloc[0]
    df.drop(index=0, inplace= True)
    if products == False:
        return df
    if products == True:
        products = df.Product.to_list()
        return df, products


def get_request_adjustment(user_request, Page_object, df_reference):
    page_name = Page_object.name
    product_row = df_reference[df_reference['Product'] == user_request]
    adjustment = product_row[page_name].to_list()[0]
    if adjustment == '':
        return None
    else:
        try:
            adjustment = float(adjustment)
        except:
            return adjustment    
        return adjustment

## Test

In [95]:
#Google sheet info
df_x_adj = get_df('x_adjustment')
df_cat = get_df('Categories')
products = df_x_adj.Product.to_list()

Pages = [Amazon, Mercado_Libre, Ebay, Best_Buy]

#Corpus
h = 0
products_from_pages = []
# user_request = products[13]
url = 'https://murmuring-plains-72181.herokuapp.com/scrap/'

# products = ['laptop hp']

#Total of 44
for user_request in products[10:11]:
    for Page in Pages:
        products_from_pages.append(request_products(user_request, Page, header=h))
        if h < 3:
            h = 0
        else:
            h +=1

    page_names = []
    for page in products_from_pages:
        if page['status'] == 200:
            page_names.append(page['names'])
        else:
            print(f"Error: {page['status']} with {page['status']}")

    if len(page_names) > 0:
        corpus = create_corpus_points(user_request, page_names, header=h)
        # Pages = [Best_Buy]
        print(f'corpus len: {len(corpus)}', end='\n\n')
        for p in range(len(Pages)):
            page_products = products_from_pages[p]
            Page = Pages[p]
            print(Page.name)
            print(str(type(page_products['prices'])))
            if str(type(page_products['prices'])) == "<class 'list'>":
                page_products['hit points'] = hit_points_info(page_products, corpus)

                ads = get_request_adjustment(user_request, Page, df_x_adj)
                print('ads', ads)

                cat = get_request_adjustment(user_request, Page, df_cat)
                print('cat', cat)
                recomended_product = add_clasification_info(page_products,
                x_adjustment=ads, category_given=cat, cheapest_dict=True, graph=False)
                print('Info of the cheapest product recomended')
                recomended_product['store'] = Page.index
                recomended_product['product'] = user_request
                for key, value in recomended_product.items():
                    print(key, ':', value)

                r = requests.post(url=url, json=recomended_product)
                request_answer = str(r.text)
                print(str(type(request_answer)))
                print('request answer:', request_answer)
                if  re.match(u"Server Error" , request_answer) is not None:
                    t=1
                    while re.match(u"Server Error" ,r.text) is not None and t < 10:
                        print('Server Error')
                        r = requests.post(url=url, json=recomended_product)
                        time.sleep(1)
                        t += 1

                        print(r.text)
                else:
                    print(r.text)
                    print('\n')
            else: print(page_products['prices'])

corpus len: 439

Amazon
<class 'list'>
ads 1.0
cat None
ah ja! GOTCHA! 399.0
Desireable
{'Desireable': [399.0, 30], 'Expensive but Requested': [None, None], "Couldn't find better": [329.0, 4], 'Emergency Case': [3495.86, 26], 'Ignored': [None, None]}
Info of the cheapest product recomended
name : Granvela® Guantes De Boxeo, ZOOBOO Guantes De Combate MMA Entrenamiento De Boxeo Mitad de los Mitones, Muay Thai, Sparring
picture : https://m.media-amazon.com/images/I/71FWWGQKLzL._AC_UL320_.jpg
url : https://www.amazon.com.mx/Granvela%C2%AE-Guantes-Combate-Entrenamiento-Sparring/dp/B01B5U4BYO/ref=sr_1_31?dchild=1&keywords=guantes+de+boxeo&qid=1601680632&sr=8-31
price : 399.0
store : 2
product : guantes de boxeo
<class 'str'>
request answer: {"name":"GranvelaÂ® Guantes De Boxeo, ZOOBOO Guantes De Combate MMA Entrenamiento De Boxeo Mitad de los Mitones, Muay Thai, Sparring","price":"399.00","url":"https://www.amazon.com.mx/Granvela%C2%AE-Guantes-Combate-Entrenamiento-Sparring/dp/B01B5U4BYO/ref